In [1]:
import warnings
warnings.filterwarnings("ignore")

from scipy import constants as cst
from pint import UnitRegistry
ureg = UnitRegistry()
ureg = UnitRegistry(autoconvert_offset_to_baseunit = True)
import sys
import multiprocessing
import subprocess
import numpy as np
import shutil
import os

path_to_code = "../generated-codes/chapter7/"
sys.path.append(path_to_code)

from MinimizeEnergy import MinimizeEnergy
from MonteCarlo import MonteCarlo

In [2]:
kB = cst.Boltzmann*ureg.J/ureg.kelvin # boltzman constant
Na = cst.Avogadro/ureg.mole # avogadro
R = kB*Na # gas constant

In [3]:
tau = 1

In [4]:
epsilon = (119.76*ureg.kelvin*kB*Na).to(ureg.kcal/ureg.mol) # kcal/mol
r_star = 3.822*ureg.angstrom # angstrom
sigma = r_star / 2**(1/6) # angstrom
N_atom = 200 # no units
m_argon = 39.948*ureg.gram/ureg.mol
T = (55 * ureg.degC).to(ureg.degK) # 55°C
volume_star = r_star**3 * Na * 2**(-0.5)
cut_off = sigma*2.5
displace_mc = sigma/5 # angstrom
volume = N_atom*volume_star*tau/Na
box_size = volume**(1/3)
folder = "./"

# Minimization
em = MinimizeEnergy(maximum_steps=100,
                    thermo_period=10,
                    dumping_period=10,
    number_atoms=[N_atom],
    epsilon=[epsilon.magnitude], 
    sigma=[sigma.magnitude],
    atom_mass=[m_argon.magnitude],
    box_dimensions=[box_size.magnitude,
                    box_size.magnitude,
                    box_size.magnitude],
    cut_off=cut_off.magnitude,
    data_folder=folder,
)
em.run()

# Equilibration run
mc1 = MonteCarlo(maximum_steps=20000,
    dumping_period=1000,
    thermo_period=1000,
    neighbor=50,
    displace_mc = displace_mc.magnitude,
    desired_temperature = T.magnitude,
    number_atoms=[N_atom],
    epsilon=[epsilon.magnitude], 
    sigma=[sigma.magnitude],
    atom_mass=[m_argon.magnitude],
    box_dimensions=[box_size.magnitude,
                    box_size.magnitude,
                    box_size.magnitude],
    initial_positions = em.atoms_positions*em.reference_distance,
    cut_off=cut_off.magnitude,
    data_folder=folder,
)
mc1.run()


step epot maxF
0 60255699339.499 2708842392734.161
10 22348323.375 52050267.097
20 250051.544 228686.296
30 6022.909 1716.278
40 612.776 100.410
50 24.976 52.448
60 -132.421 10.949
70 -215.914 18.855
80 -268.185 19.227
90 -286.864 0.826
100 -296.208 1.630
step  N     T (K)     p (atm)   V (A3)    Ep (kcal/mol) Ek (kcal/mol) dens (g/cm3) 
0     200   3.28e+02  3.62e+03  7.9e+03   -2.96e+02     0.0           1.68         
1000  200   3.28e+02  6.93e+03  7.9e+03   -2.19e+02     0.0           1.68         
2000  200   3.28e+02  7.86e+03  7.9e+03   -1.97e+02     0.0           1.68         
3000  200   3.28e+02  8.5e+03   7.9e+03   -1.82e+02     0.0           1.68         
4000  200   3.28e+02  8.26e+03  7.9e+03   -1.87e+02     0.0           1.68         
5000  200   3.28e+02  8.93e+03  7.9e+03   -1.71e+02     0.0           1.68         
6000  200   3.28e+02  8.88e+03  7.9e+03   -1.72e+02     0.0           1.68         
7000  200   3.28e+02  9.25e+03  7.9e+03   -1.64e+02     0.0           1.

In [6]:

# Short production run
mc2 = MonteCarlo(maximum_steps=10000,
    dumping_period=100,
    thermo_period=100,
    neighbor=50,
    displace_mc = displace_mc.magnitude,
    desired_temperature = T.magnitude,
    number_atoms=[N_atom],
    epsilon=[epsilon.magnitude], 
    sigma=[sigma.magnitude],
    atom_mass=[m_argon.magnitude],
    box_dimensions=[box_size.magnitude,
                    box_size.magnitude,
                    box_size.magnitude],
    initial_positions = mc1.atoms_positions*mc1.reference_distance,
        cut_off=cut_off.magnitude,
        data_folder=folder,
    )
mc2.run()

step  N     T (K)     p (atm)   V (A3)    Ep (kcal/mol) Ek (kcal/mol) dens (g/cm3) 
0     200   3.28e+02  8.79e+03  7.9e+03   -1.74e+02     0.0           1.68         
100   200   3.28e+02  8.7e+03   7.9e+03   -1.76e+02     0.0           1.68         
200   200   3.28e+02  8.85e+03  7.9e+03   -1.72e+02     0.0           1.68         
300   200   3.28e+02  8.93e+03  7.9e+03   -1.7e+02      0.0           1.68         
400   200   3.28e+02  8.91e+03  7.9e+03   -1.71e+02     0.0           1.68         
500   200   3.28e+02  9.11e+03  7.9e+03   -1.66e+02     0.0           1.68         
600   200   3.28e+02  8.88e+03  7.9e+03   -1.72e+02     0.0           1.68         
700   200   3.28e+02  8.66e+03  7.9e+03   -1.76e+02     0.0           1.68         
800   200   3.28e+02  8.85e+03  7.9e+03   -1.72e+02     0.0           1.68         
900   200   3.28e+02  9e+03     7.9e+03   -1.68e+02     0.0           1.68         
1000  200   3.28e+02  8.93e+03  7.9e+03   -1.71e+02     0.0           1.68  